## Developing Clinical Conversational AI for patients after TBI

- Script name: Sihan_Tu_LLM_TBI.ipynb
- Created on: 07/10/2025
- Author: Sihan Tu
- Purpose:
- Version: 3.0
- Notes:

#### The Unified Interface for LLMs from: [OpenRouter.AI](https://openrouter.ai/)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ⚠️ Important Warnings

- **Always check your global configuration (`CONFIG`) before running the experiment.**  
  Ensure that:
  - All folder paths are correct.
  - Prompts are loaded correctly.
  - Model names, temperatures, and top_p values are set as intended.
  - `num_runs` is configured according to your needs.

- **Always verify the `run_id` before starting a new run.**  
  - `run_id` determines the naming of `.txt` files and Excel sheets.
  - Restarting a notebook without updating `run_id` may **overwrite previous results**.
  - If you want to continue from previous runs, set `run_id` to the next available number.

- **Tip:** Double-check your output directories:
  - Transcripts: `/content/drive/MyDrive/Sihan_Tu_LLM_TBI/Result/Transcript`
  - Results Excel: `/content/drive/MyDrive/Sihan_Tu_LLM_TBI/Result/Data`


### Libraries

In [ ]:
import glob
import os
import itertools
import time
import pandas as pd
import re
import json

### Global Configuration

In [ ]:
# Function to load all prompts from a folder
def load_prompts_from_folder(folder_path):
    file_paths = sorted(glob.glob(os.path.join(folder_path, "*.txt")))
    return [
        (os.path.basename(f), open(f, "r", encoding="utf-8").read())
        for f in file_paths
    ]

# Folders for each agent
doctor_folder = "/content/drive/MyDrive/Sihan_Tu_LLM_TBI/Prompt/Doctor"
patient_folder = "/content/drive/MyDrive/Sihan_Tu_LLM_TBI/Prompt/Patient"
report_folder = "/content/drive/MyDrive/Sihan_Tu_LLM_TBI/Prompt/Report"
evaluation_folder = "/content/drive/MyDrive/Sihan_Tu_LLM_TBI/Prompt/Evaluation"

CONFIG = {
    "models": {
        "doctor": ["openai/gpt-5.2"],
        "patient": ["openai/gpt-5.2"],
        "report": ["openai/gpt-5.2"],
        "evaluation": ["openai/gpt-5.2"],
    },

    "prompts": {
        "doctor": load_prompts_from_folder(doctor_folder),
        "patient": load_prompts_from_folder(patient_folder),
        "report": load_prompts_from_folder(report_folder),
        "evaluation": load_prompts_from_folder(evaluation_folder),
    },

    "parameters": {
        "doctor": {
            "temperature": [0.85],
            "top_p": [1]
        },

        "patient": {
            "temperature": [1],
            "top_p": [1]
        },

        "report":  {
            "temperature": [0.85],
            "top_p": [1]
        },

        "evaluation": {
            "temperature": [1],
            "top_p": [1]
        },
    },

    "run_id": 1,      # Set to next available number to avoid overwriting previous results
    "repeatCount": 1  # Number of times to repeat this full combination

}


In [ ]:
CONFIG["models"]

{'doctor': ['openai/gpt-5.2'],
 'patient': ['openai/gpt-5.2'],
 'report': ['openai/gpt-5.2'],
 'evaluation': ['openai/gpt-5.2']}

In [ ]:
CONFIG["prompts"]["doctor"]

[('doctor_prompt_4.txt',
  'You are an expert TBI neurologist assistant conducting a detailed head injury consultation.\nYour ONLY task is to take a full conversational history.\nYou must NEVER diagnose or give medical advice.\n\n## CONVERSATION STATE\n\n- opening_completed: false\n- demographics_completed: false\n\n## CONSULTATION APPROACH\n\n### Opening\n\nAt the start of the consultation, if opening_completed is false, begin with an empathetic introduction using the exact wording below.\n\nThis introduction should be asked only once per consultation and must not be repeated after opening_completed becomes true.\n\nOpening wording (use verbatim, once only):\n\n"Thank you for speaking with me today. I\'m here to understand what happened and how you\'ve been since your injury. Can you tell me in your own words what happened?"\n\nAfter asking this opening question, set opening_completed = true.\n\n### Core Demographics and Background\n\nIf opening_completed is true AND demographics_comp

### Client Initialization

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=""
)

### LLM Runner

In [ ]:
def run_llm(model, messages, params):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=params["temperature"],
        top_p=params["top_p"],
    )
    return response.choices[0].message.content


In [ ]:
def build_transcript(messages):
    transcript_text = ""
    for msg in messages:
        # Map roles to desired names
        role_name = ""
        if msg["role"] == "system":
            role_name = "AI TBI Assistant"
        elif msg["role"] == "user":
            role_name = "Patient"
        else:
            role_name = msg["role"].capitalize()

        transcript_text += f"{role_name}:\n{msg['content']}\n\n"
    return transcript_text

In [ ]:
def run_conversation(
    doctor_model, doctor_prompt,
    patient_model, patient_prompt,
    params
):
    messages = []
    messages_pat = []
    consultation_complete = False
    total_start_time = time.time()
    turn_count = 0

    doctor_total_time = 0.0
    patient_total_time = 0.0

    while not consultation_complete:
        turn_count += 1

        # ---- Doctor turn ----
        doctor_input = [
            {"role": "system", "content": doctor_prompt},
            *messages
        ]

        t0 = time.time()
        doctor_msg = run_llm(
            model=doctor_model,
            messages=doctor_input,
            params=params["doctor"]
        )
        t1 = time.time()
        doctor_duration = (t1 - t0)
        doctor_total_time += doctor_duration

        print(f"\nAI TBI Assistant (T{turn_count}: {doctor_duration:.2f} seconds):\n{doctor_msg}\n")
        messages.append({"role": "system", "content": doctor_msg})
        messages_pat.append({"role": "user", "content": doctor_msg})

        # ---- stop phrase ----
        msg_lower = doctor_msg.lower()
        if ("urgent medical assessment" in msg_lower or
            "report for the treating neurologist" in msg_lower):
            consultation_complete = True
            break

        # ---- Patient turn ----
        patient_input = [
            {"role": "system", "content": patient_prompt},
            *messages_pat
        ]

        t0 = time.time()
        patient_msg = run_llm(
            model=patient_model,
            messages=patient_input,
            params=params["patient"]
        )
        t1 = time.time()
        patient_duration = (t1 - t0)
        patient_total_time += patient_duration

        print(f"\nPatient (T{turn_count}: {patient_duration:.2f} seconds):\n{patient_msg}\n")
        messages.append({"role": "user", "content": patient_msg})
        messages_pat.append({"role": "system", "content": patient_msg})

    # ---- Finished ----
    total_duration = time.time() - total_start_time

    avg_doctor_time = doctor_total_time / turn_count if turn_count else 0
    avg_patient_time = patient_total_time / turn_count if turn_count else 0

    transcript_text = build_transcript(messages)

    return {
        "messages": messages,
        "transcript_text": transcript_text,
        "turn_count": turn_count,
        "doctor_total_time": doctor_total_time,
        "patient_total_time": patient_total_time,
        "avg_doctor_time": avg_doctor_time,
        "avg_patient_time": avg_patient_time,
        "total_duration": total_duration
    }


In [ ]:
def run_report(report_model, report_prompt, transcript_text, params):
    messages = [
        {"role": "system", "content": report_prompt},
        {"role": "user", "content": transcript_text}
    ]
    report_text = run_llm(
        model=report_model,
        messages=messages,
        params=params["report"]
    )

    print(f"\n=== REPORT ===\n{report_text}\n")

    return report_text


In [ ]:
def run_evaluation(evaluation_model, evaluation_prompt, transcript_text, report_text, params):
    eval_input = (
        "=== TRANSCRIPT ===\n" +
        transcript_text +
        "\n\n=== REPORT ===\n" +
        report_text +
        "\n"
    )
    messages = [
        {"role": "system", "content": evaluation_prompt},
        {"role": "user", "content": eval_input}
    ]
    evaluation_text = run_llm(
        model=evaluation_model,
        messages=messages,
        params=params["evaluation"]
    )

    # Print the evaluation
    print(f"\n=== EVALUATION ===\n{evaluation_text}\n")

    return evaluation_text


In [ ]:
# Folders for saving outputs
txt_output_dir = "/content/drive/MyDrive/Sihan_Tu_LLM_TBI/Result/Transcript"
os.makedirs(txt_output_dir, exist_ok=True)

data_output_dir = "/content/drive/MyDrive/Sihan_Tu_LLM_TBI/Result/Data"
os.makedirs(data_output_dir, exist_ok=True)

results = []
run_id = CONFIG["run_id"]

for (doctor_prompt_file, doctor_prompt), doctor_model, doctor_temp, doctor_top_p in itertools.product(
    CONFIG["prompts"]["doctor"],
    CONFIG["models"]["doctor"],
    CONFIG["parameters"]["doctor"]["temperature"],
    CONFIG["parameters"]["doctor"]["top_p"],
):

    for (patient_prompt_file, patient_prompt), patient_model, patient_temp, patient_top_p in itertools.product(
        CONFIG["prompts"]["patient"],
        CONFIG["models"]["patient"],
        CONFIG["parameters"]["patient"]["temperature"],
        CONFIG["parameters"]["patient"]["top_p"],
    ):

        for (report_prompt_file, report_prompt), report_model, report_temp, report_top_p in itertools.product(
            CONFIG["prompts"]["report"],
            CONFIG["models"]["report"],
            CONFIG["parameters"]["report"]["temperature"],
            CONFIG["parameters"]["report"]["top_p"],
        ):

            for (eval_prompt_file, eval_prompt), eval_model, eval_temp, eval_top_p in itertools.product(
                CONFIG["prompts"]["evaluation"],
                CONFIG["models"]["evaluation"],
                CONFIG["parameters"]["evaluation"]["temperature"],
                CONFIG["parameters"]["evaluation"]["top_p"],
            ):

                for repeat_index in range(CONFIG["repeatCount"]):

                    print("\n\n==============================")
                    print(f"STARTING RUN_{run_id:05d}")
                    print("==============================\n")

                    params = {
                        "doctor": {"temperature": doctor_temp, "top_p": doctor_top_p},
                        "patient": {"temperature": patient_temp, "top_p": patient_top_p},
                        "report": {"temperature": report_temp, "top_p": report_top_p},
                        "evaluation": {"temperature": eval_temp, "top_p": eval_top_p},
                    }

                    # 1. Conversation
                    convo = run_conversation(
                        doctor_model,
                        doctor_prompt,
                        patient_model,
                        patient_prompt,
                        params
                    )
                    transcript_text = convo["transcript_text"]

                    # 2. Report
                    report_text = run_report(
                        report_model,
                        report_prompt,
                        transcript_text,
                        params
                    )

                    # 3. Evaluation
                    evaluation_text = run_evaluation(
                        eval_model,
                        eval_prompt,
                        transcript_text,
                        report_text,
                        params
                    )

                    # --- Save full transcript + report + evaluation to .txt ---
                    run_name = f"RUN_{run_id:05d}"
                    txt_path = os.path.join(txt_output_dir, f"{run_name}.txt")
                    with open(txt_path, "w", encoding="utf-8") as f:
                        # --- Run name ---
                        f.write(f"=== {run_name} ===\n\n")

                        # --- Prompts ---
                        f.write("=== DOCTOR PROMPT ===\n")
                        f.write(doctor_prompt + "\n\n")

                        f.write("=== PATIENT PROMPT ===\n")
                        f.write(patient_prompt + "\n\n")

                        f.write("=== REPORT PROMPT ===\n")
                        f.write(report_prompt + "\n\n")

                        f.write("=== EVALUATION PROMPT ===\n")
                        f.write(eval_prompt + "\n\n")

                        # --- Generated content ---
                        f.write("=== TRANSCRIPT ===\n")
                        f.write(transcript_text + "\n\n")
                        f.write("=== REPORT ===\n")
                        f.write(report_text + "\n\n")
                        f.write("=== EVALUATION ===\n")
                        f.write(evaluation_text)

                    # --- Append row to results list ---
                    row = {
                        "run_id": run_name,
                        "doctor_model": doctor_model,
                        "doctor_prompt_file": doctor_prompt_file,
                        "doctor_temperature": doctor_temp,
                        "doctor_top_p": doctor_top_p,

                        "patient_model": patient_model,
                        "patient_prompt_file": patient_prompt_file,
                        "patient_temperature": patient_temp,
                        "patient_top_p": patient_top_p,

                        "report_model": report_model,
                        "report_prompt_file": report_prompt_file,
                        "report_temperature": report_temp,
                        "report_top_p": report_top_p,

                        "evaluation_model": eval_model,
                        "evaluation_prompt_file": eval_prompt_file,
                        "evaluation_temperature": eval_temp,
                        "evaluation_top_p": eval_top_p,

                        "turn_count": convo["turn_count"],
                        "doctor_total_time": convo["doctor_total_time"],
                        "patient_total_time": convo["patient_total_time"],
                        "avg_doctor_time": convo["avg_doctor_time"],
                        "avg_patient_time": convo["avg_patient_time"],
                        "total_duration": convo["total_duration"],

                        "txt_log_path": txt_path
                    }

                    # --- After evaluation_text is generated ---
                    evaluation_data = json.loads(evaluation_text)

                    # --- Extract scores from evaluation ---
                    score_dict = {}
                    for domain in evaluation_data.get("domains", []):
                        domain_id = domain["id"]
                        score_dict[f"{domain_id}_score"] = domain.get("score")
                        score_dict[f"{domain_id}_comment"] = domain.get("comment")

                    # --- Include overall total and rating ---
                    score_dict["total_score"] = evaluation_data.get("scoring_rules_applied", {}).get("total_score")
                    score_dict["overall_rating"] = evaluation_data.get("scoring_rules_applied", {}).get("overall_rating")
                    score_dict["summary_comment"] = evaluation_data.get("summary_comment")

                    # --- Add score_dict to your row ---
                    row.update(score_dict)

                    results.append(row)

                    # --- Save results to Excel after each run ---
                    excel_path = os.path.join(data_output_dir, f"{run_name}.xlsx")
                    pd.DataFrame([row]).to_excel(excel_path, index=False)

                    run_id += 1




STARTING RUN_00031


AI TBI Assistant (T1: 2.09 seconds):
Thank you for speaking with me today. I'm here to understand what happened and how you've been since your injury. Can you tell me in your own words what happened?


Patient (T1: 6.06 seconds):
I’m getting these spins when I roll over or look up—since I bumped my head.

About three weeks ago I was cycling fairly slowly and I skidded on a bit of wet path. I fell off and, even though I had a helmet on, I knocked the right side of my head—around the parietal area—on the ground. I didn’t black out and I remember everything, but I had a mild headache for the rest of that day.

A couple of days after that, I started getting these sudden, really intense “room spinning” sensations, but only with certain movements—like rolling over in bed, especially to the right, or looking up to a high shelf, or bending down. They last maybe 10–20 seconds, then settle if I keep still, but they can make me feel nauseated. Between episodes I feel a bit 

AuthenticationError: Error code: 401 - {'error': {'message': 'User not found.', 'code': 401}}